In [1]:
import tensorflow as tf
import numpy as np
import json
import os
import csv
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
ML_Summer_School_ID = os.getenv('ML_Summer_School_ID')
print("Your Sudent ID is: " + ML_Summer_School_ID)


Your Sudent ID is: ML-062


In [4]:
BATCH_SIZE  = 32
input_img_size = (128,128)
input_shape = (128,128,3)


test_dir = r'C:\Users\aungh\Projects\WAI ML\PML-0006-aunghtet-haw\6_deep_learning\CNN\cats_and_dogs_filtered\test'
model_path = f"./binary_classification/{ML_Summer_School_ID}_model.h5"
class_index_path = f'./binary_classification/{ML_Summer_School_ID}_class_indices.json'
mark_path = "./binary_classification/marks.txt"


In [5]:
model = tf.keras.models.load_model(model_path)
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 16, 16, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 8, 8, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     8,388,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,111,236 (50.02 MB)

 Trainable params: 13,109,314 (50.01 MB)

 Non-trainable params: 1,920 (7.50 KB)

 Optimizer params: 2 (12.00 B)

None


In [6]:
# Load class indices from the JSON file
with open(class_index_path, 'r') as f:
    class_indices = json.load(f)
print("Class indices loaded:", class_indices)


Class indices loaded: {'cats': 0, 'dogs': 1}


In [7]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=input_img_size,
    batch_size=BATCH_SIZE,
    shuffle=False,  # Important: preserve label order for evaluation
    label_mode='int'  # returns integer labels
)

print("Test dataset classes:", test_ds.class_names)
print("Loaded class_indices:", class_indices)

# Normalize test dataset to match training rescaling
test_ds = test_ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))

Found 248 files belonging to 2 classes.
Test dataset classes: ['cats', 'dogs']
Loaded class_indices: {'cats': 0, 'dogs': 1}


In [8]:
# === Predict ===
y_true = []
y_pred = []


for images, labels in test_ds:
    predictions = model.predict(images)

    # If binary classification
    if predictions.shape[1] == 1:
        preds = (predictions > 0.5).astype(int).flatten()
    else:  # multi-class
        preds = np.argmax(predictions, axis=1)

    y_true.extend(labels.numpy())
    y_pred.extend(preds)

# === Evaluate Accuracy ===
y_true = np.array(y_true)
y_pred = np.array(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 735ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 837ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 734ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 727ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 723ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 717ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 864ms/step


In [9]:
accuracy = np.mean(y_true == y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 82.26%


In [10]:
with open(mark_path, "w") as f:
    f.write(str(int(accuracy * 100)))